### **<u>DATA PREPROCESSING</u>**

### ***1. Dataset Conversion from CSV to TSV***

This code imports the pandas library and defines a function to convert the two dataset CSV files into TSV files. It reads each CSV file into a pandas DataFrame, saves it as a TSV file.

In [1]:
import pandas as pd
import os

# Function to convert CSV to TSV with custom output file names
def convert_csv_to_tsv():
    # Define the input CSV files and their corresponding output TSV file names
    csv_files = {
        'temperature.csv': 'Temperature.tsv',
        'BinSize.csv': 'Stations.tsv'
    }

    # Iterate over the CSV files and convert them to TSV
    for csv_file, tsv_file in csv_files.items():
        try:
            # Construct the full path to the CSV file
            csv_path = os.path.join(os.getcwd(), csv_file)
            
            # Read the CSV file
            df = pd.read_csv(csv_path)
            
            # Construct the full path to the TSV file
            tsv_path = os.path.join(os.getcwd(), tsv_file)
            
            # Save the DataFrame as a TSV file
            df.to_csv(tsv_path, sep='\t', index=False)
            print(f'Converted {csv_file} to {tsv_file}')
            
            # Optionally, read the TSV file back to verify
            df_tsv = pd.read_csv(tsv_path, sep='\t')
        except FileNotFoundError:
            print(f'Error: The file {csv_file} was not found.')
        except Exception as e:
            print(f'An error occurred while processing {csv_file}: {e}')

# Call the function to perform the conversion
convert_csv_to_tsv()

Converted temperature.csv to Temperature.tsv
Converted BinSize.csv to Stations.tsv


### Why TSV is Better:
- TSV files use tab as a delimiter, reducing issues caused by commas in data fields.
- TSV format is simpler and less prone to errors when fields contain commas, making it more robust for certain datasets.
- TSV files are easy to parse, especially in Unix-based systems where tools like cut, awk, and sed are commonly used.
- They ensure better data integrity when working with complex textual data compared to CSV files.



### ***2. Temperature Dataset Cleaning***

This code below reflects modifications made to the original temperature dataset for Ann Arbor, Michigan. The result is a cleaned and organized dataset, saved as a new file, optimized for creating graphs and visualizations


In [2]:
import pandas as pd
import os 
from datetime import datetime

# Ask the user for the input file path
file_path = os.path.join(os.getcwd(), 'Temperature.tsv')

# Load the .tsv file into a DataFrame
try:
    df = pd.read_csv(file_path, sep='\t')
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")
    exit()

# Print the number of rows before any changes
print(f"Number of rows before any changes: {len(df)}")

# Function to check and convert dates from mm-dd-yyyy to dd-mm-yyyy
def convert_date(date_str):
    try:
        # Try parsing the date in mm-dd-yyyy format
        date_obj = datetime.strptime(date_str, '%m-%d-%Y')
        # If successful, convert it to dd-mm-yyyy format
        return date_obj.strftime('%d-%m-%Y')
    except ValueError:
        # If parsing fails, return the original date (it's not in mm-dd-yyyy format)
        return date_str

# Apply the conversion function to the 'Date' column
df['Date'] = df['Date'].apply(convert_date)

# Check if any dates were converted (i.e., originally in mm-dd-yyyy format)
converted_dates = df[df['Date'].str.contains(r'\d{2}-\d{2}-\d{4}')]
if not converted_dates.empty:
    print("The following dates were in mm-dd-yyyy format and have been converted to dd-mm-yyyy format:")
    print(converted_dates)
else:
    print("No dates were found in mm-dd-yyyy format.")

# Convert the 'Date' column to datetime format for proper sorting
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)

# Clean up rows where the date is February 29th (leap days)
leap_days = df[df['Date'].dt.strftime('%d-%m') == '29-02']
if not leap_days.empty:
    print("\nThe following rows contain leap days (February 29th) and will be removed:")
    print(leap_days)
    df = df[df['Date'].dt.strftime('%d-%m') != '29-02']
else:
    print("\nNo leap days (February 29th) were found in the DataFrame.")

# Check if any leap days remain in the DataFrame
remaining_leap_days = df[df['Date'].dt.strftime('%d-%m') == '29-02']
if not remaining_leap_days.empty:
    print("\nWarning: The following leap days (February 29th) still exist in the DataFrame:")
    print(remaining_leap_days)
else:
    print("\nNo leap days (February 29th) remain in the DataFrame.")

# Convert temperatures from tenths of degrees Celsius to degrees Celsius
df['Data_Value'] = df['Data_Value'] / 10

# Rename the 'Data_Value' column to 'temperature(°C)'
df.rename(columns={'Data_Value': 'Temperature(°C)'}, inplace=True)

# Print the number of rows after all changes
print(f"\nNumber of rows after all changes: {len(df)}")

# Print the maximum and minimum values in the 'temperature(°C)' column
max_temp = df['Temperature(°C)'].max()
min_temp = df['Temperature(°C)'].min()
print(f"\nMaximum temperature(°C) across all rows: {max_temp}")
print(f"Minimum temperature(°C) across all rows: {min_temp}")

# Sort the DataFrame by the 'Date' column in ascending order
df_sorted = df.sort_values(by='Date')

# Check if the DataFrame is sorted by the 'Date' column
is_sorted = df_sorted['Date'].is_monotonic_increasing

# Print the result of the check
if is_sorted:
    print("\nThe DataFrame is sorted by the 'Date' column in ascending order.")
else:
    print("\nThe DataFrame is NOT sorted by the 'Date' column.")

# Display the first 5 rows of the sorted DataFrame
print("\nFirst 5 rows of the sorted DataFrame:")
print(df_sorted.head())

# Save the modified DataFrame to a new .tsv file
output_file_path = 'Temp_Cleaned.tsv'
df_sorted.to_csv(output_file_path, sep='\t', index=False)
print(f"\nThe modified DataFrame has been saved as '{output_file_path}'.")

Number of rows before any changes: 165085
No dates were found in mm-dd-yyyy format.

The following rows contain leap days (February 29th) and will be removed:
                 ID       Date Element  Data_Value
18712   USC00205563 2012-02-29    TMAX          39
18716   USC00200842 2012-02-29    TMIN         -56
18729   USC00205563 2012-02-29    TMIN          -6
18730   USC00200842 2012-02-29    TMAX          61
18812   USC00208080 2012-02-29    TMIN         -44
...             ...        ...     ...         ...
154086  USC00208972 2008-02-29    TMIN        -128
154089  USC00208972 2008-02-29    TMAX         -39
161046  USC00203712 2008-02-29    TMAX         -33
161054  USC00203712 2008-02-29    TMIN        -111
164153  USC00205050 2008-02-29    TMAX          22

[83 rows x 4 columns]

No leap days (February 29th) remain in the DataFrame.

Number of rows after all changes: 165002

Maximum temperature(°C) across all rows: 40.6
Minimum temperature(°C) across all rows: -34.3

The DataFrame 

#### Changes Made with Code Snippets:

**1. Date Format Conversion:**

In [ ]:
df['Date'] = df['Date'].apply(convert_date)

*This line applies a function that converts dates from `mm-dd-yyyy` to `dd-mm-yyyy`.*

**2. Leap Day Removal:**


In [ ]:
df = df[df['Date'].dt.strftime('%d-%m') != '29-02']

*Rows with February 29th are removed to avoid inconsistencies across different years.*

**3. Temperature Conversion:**

In [ ]:
df['Data_Value'] = df['Data_Value'] / 10

*Temperature values are converted from tenths of degrees Celsius to degrees Celsius.*

**4. Column Renaming:**

In [ ]:
df.rename(columns={'Data_Value': 'Temperature(°C)'}, inplace=True)

*Renames the column for better understanding.*

**5. Sorting:**

In [ ]:
df_sorted = df.sort_values(by='Date')

*Ensures the data is arranged in chronological order for better analysis*

**6. File Output:**

In [ ]:
df_sorted.to_csv('Temp_Cleaned.tsv', sep='\t', index=False)

*Saves the cleaned Temperature dataset as a new TSV file for future use.*



### ***3. Weather Station Dataset Cleaning***

This code below reflects modifications made to the original Weather stations dataset. The result is a cleaned and organized dataset, saved as a new file, optimized for creating graphs and visualizations

In [3]:
import pandas as pd
import os
from geopy.distance import geodesic

# Ask the user for the path to the first .tsv file
file_path_1 = os.path.join(os.getcwd(), 'Stations.tsv')

# Load the first .tsv file into a DataFrame
try:
    df1 = pd.read_csv(file_path_1, sep='\t')
except FileNotFoundError:
    print("First file not found. Please check the file path and try again.")
    exit()

# Keep only the 1st, 2nd, 3rd, and 6th columns
columns_to_keep = df1.columns[[0, 1, 2, 5]]  # Indices 0, 1, 2, and 5 correspond to 1st, 2nd, 3rd, and 6th columns
df1_filtered = df1[columns_to_keep].copy()  # Explicitly create a copy to avoid the warning

# Round the LATITUDE and LONGITUDE columns to 6 decimal places using .loc
df1_filtered.loc[:, 'LATITUDE'] = df1_filtered['LATITUDE'].round(6)
df1_filtered.loc[:, 'LONGITUDE'] = df1_filtered['LONGITUDE'].round(6)

# Ask the user for the path to the second .tsv file
file_path_2 = os.path.join(os.getcwd(), 'Temperature.tsv')

# Load the second .tsv file into a DataFrame
try:
    df2 = pd.read_csv(file_path_2, sep='\t')
except FileNotFoundError:
    print("Second file not found. Please check the file path and try again.")
    exit()

# Filter the first DataFrame to keep only rows where the ID exists in the second DataFrame
df1_filtered = df1_filtered[df1_filtered['ID'].isin(df2['ID'])]

# Print the number of rows that match
num_matching_rows = len(df1_filtered)
print(f"\nNumber of rows where ID exists in the second file: {num_matching_rows}")

# Define Ann Arbor coordinates
ann_arbor_coords = (42.2808, -83.7430)

# Filter stations within 100 km of Ann Arbor
df1_filtered = df1_filtered[
    df1_filtered.apply(
        lambda row: geodesic((row['LATITUDE'], row['LONGITUDE']), ann_arbor_coords).km <= 100,
        axis=1
    )
]

# Print the number of rows that satisfy the 100 km radius condition
num_nearby_stations = len(df1_filtered)
print(f"\nNumber of stations within 100 km of Ann Arbor: {num_nearby_stations}")

# Save the final filtered DataFrame to a new .tsv file
output_file_path = 'W-Stations_Cleaned.tsv'
df1_filtered.to_csv(output_file_path, sep='\t', index=False)
print(f"\nThe final filtered DataFrame has been saved as '{output_file_path}'.")




Number of rows where ID exists in the second file: 24

Number of stations within 100 km of Ann Arbor: 24

The final filtered DataFrame has been saved as 'W-Stations_Cleaned.tsv'.


#### Changes Made with Code Snippets:

**1. Unnecessary data cleaning:**


In [ ]:
columns_to_keep = df1.columns[[0, 1, 2, 5]]
df1_filtered = df1[columns_to_keep]

*Keeps only essential columns: `ID`, `LATITUDE`, `LONGITUDE`, and `NAME` for relevance and to reduce dataset size.*

**2. Numerical Consistency:**

In [ ]:
df1_filtered['LATITUDE'] = df1_filtered['LATITUDE'].round(6)
df1_filtered['LONGITUDE'] = df1_filtered['LONGITUDE'].round(6)

*Rounds coordinates to 6 decimal places for consistency and precision.*

**3. Filter Dataset:**

In [ ]:
file_path_2 = input("\nEnter the path to your second .tsv file: ")
df1_filtered = df1_filtered[df1_filtered['ID'].isin(df2['ID'])]

*Asks for The temperature dataset file and filters the dataset to keep only weather stations present in the Temp_Cleaned.tsv dataset by matching `ID`.*

**4. Geospatial Filtering:**

In [ ]:
ann_arbor_coords = (42.2808, -83.7430)

df1_filtered = df1_filtered[
    df1_filtered.apply(
        lambda row: geodesic((row['LATITUDE'], row['LONGITUDE']), ann_arbor_coords).km <= 100,
        axis=1
    )
]

*Calculates distances from Ann Arbor coordinates and filter stations within a 100 km radius.*

**4. File Output:**

In [ ]:
df1_filtered.to_csv('W-Stations_Cleaned.tsv', sep='\t', index=False)


*Saves the cleaned weather station dataset new TSV file for future use.*